# Preppin' Data
## 2024: Week 11 - 13 months in a year
**Created by:** Jenny Martin | [Challenge Link](https://preppindata.blogspot.com/2024/03/2024-week-11-13-months-in-year.html)

Are you familiar with the argument that there should be 13 months in the year instead of 12? <br>
13 months, exactly 4 weeks long, would account for 364 days of the year and then we'd just have an extra 1 to deal with (or 2 on a leap year!)  <br> <br>

Naturally, my mind goes to building a workflow that let's me investigate which dates would change months.

In [25]:
# Input the csv file
import pandas as pd
date_table = pd.read_csv("2024W11 Input.csv").sort_index()
date_table

,Date
0,1st January
1,31st December


In [26]:
date_table["Date"] = date_table["Date"].str.replace(r"(st|nd|rd|th)", "", regex=True)
date_table["Date"] = date_table["Date"] + " 2024"
date_table

,Date
0,1 January 2024
1,31 December 2024


In [27]:
date_table["Date"] = pd.to_datetime(date_table["Date"], format="%d %B %Y")
date_table

,Date
0,2024-01-01
1,2024-12-31


In [28]:
# Create a row for each day of the year

start_date = date_table["Date"].min()
end_date = date_table["Date"].max()
full_year = pd.date_range(start=start_date, end=end_date, freq='D')

date_scaffold = pd.DataFrame({"Date": full_year})
date_scaffold

,Date
0,2024-01-01
1,2024-01-02
2,2024-01-03
3,2024-01-04
4,2024-01-05
...,...
361,2024-12-27
362,2024-12-28
363,2024-12-29
364,2024-12-30


In [29]:
# Calculate the new months of the year such that the first 28 days of the month are month 1, the next 28 days are month 2, etc
# This will give you 14 months, with the 14th month containing just 2 days

date_scaffold["Day Number"] = date_scaffold["Date"].dt.dayofyear
date_scaffold.head()

,Date,Day Number
0,2024-01-01,1
1,2024-01-02,2
2,2024-01-03,3
3,2024-01-04,4
4,2024-01-05,5


In [30]:
import numpy as np

date_scaffold["New Month"] = np.ceil(date_scaffold["Day Number"] / 28).astype(int)
date_scaffold

,Date,Day Number,New Month
0,2024-01-01,1,1
1,2024-01-02,2,1
2,2024-01-03,3,1
3,2024-01-04,4,1
4,2024-01-05,5,1
...,...,...,...
361,2024-12-27,362,13
362,2024-12-28,363,13
363,2024-12-29,364,13
364,2024-12-30,365,14


In [31]:
# Create a new date with the format:
# New day of the month / New month / 2024
# e.g. 20/11/2024 becomes 17/12/2024

date_scaffold["New Day"] = (date_scaffold["Day Number"] - 1) % 28 + 1
date_scaffold

,Date,Day Number,New Month,New Day
0,2024-01-01,1,1,1
1,2024-01-02,2,1,2
2,2024-01-03,3,1,3
3,2024-01-04,4,1,4
4,2024-01-05,5,1,5
...,...,...,...,...
361,2024-12-27,362,13,26
362,2024-12-28,363,13,27
363,2024-12-29,364,13,28
364,2024-12-30,365,14,1


In [36]:
date_scaffold["New Date"] = "2024-" + date_scaffold["New Month"].astype(str).str.zfill(2) + "-" + date_scaffold["New Day"].astype(str).str.zfill(2)
date_scaffold

,Date,Day Number,New Month,New Day,New Date
0,2024-01-01,1,1,1,2024-01-01
1,2024-01-02,2,1,2,2024-01-02
2,2024-01-03,3,1,3,2024-01-03
3,2024-01-04,4,1,4,2024-01-04
4,2024-01-05,5,1,5,2024-01-05
...,...,...,...,...,...
361,2024-12-27,362,13,26,2024-13-26
362,2024-12-28,363,13,27,2024-13-27
363,2024-12-29,364,13,28,2024-13-28
364,2024-12-30,365,14,1,2024-14-01


In [49]:
# Filter the data to only contain dates for which the month has changed in the new system

date_scaffold["Old Month"] = date_scaffold["Date"].astype(str).str.split("-").str.get(1)
output = date_scaffold[date_scaffold["Old Month"].astype(str).str.zfill(2) != date_scaffold["New Month"].astype(str).str.zfill(2)]
output.pop("Day Number")
output.pop("New Month")
output.pop("Old Month")
output.pop("New Day")

,Date,New Date
28,2024-01-29,2024-02-01
29,2024-01-30,2024-02-02
30,2024-01-31,2024-02-03
56,2024-02-26,2024-03-01
57,2024-02-27,2024-03-02
...,...,...
361,2024-12-27,2024-13-26
362,2024-12-28,2024-13-27
363,2024-12-29,2024-13-28
364,2024-12-30,2024-14-01


## Output
190 rows (191 including headers)<br><br>
**2 fields:**<br>
- Date
- New Date

In [ ]:
# Output the data
output

In [50]:
# Generating csv output file
output.to_csv("output-202411.csv", index=False)